In [ ]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
def load_plant_point_cloud(file_path):
    data = np.loadtxt(file_path, usecols=(0, 1, 2))
    return data


In [ ]:
##############################
# 1. Plant Data Loading      #
##############################

file_path = r"..\Sorghum Plants Labeled 3D LiDAR Point Cloud Data\labeled_plant01.txt"
plant_points = load_plant_point_cloud(file_path)  # (N, 3)
plant_labels = np.ones((plant_points.shape[0], 1))  # label=1 for plant

##############################
# 2. Plant Bounding Box      #
##############################

min_pt = np.min(plant_points, axis=0)  # [x_min, y_min, z_min]
max_pt = np.max(plant_points, axis=0)  # [x_max, y_max, z_max]
bbox_size = max_pt - min_pt            # [dx, dy, dz]
bbox_center = (min_pt + max_pt) / 2.0

##############################
# 3. Derive Synthetic Sizes  #
##############################
pot_diameter = 1.5 * max(bbox_size[0], bbox_size[1])
pot_height = 1.5 * bbox_size[2]
soil_thickness = 0.05 * pot_height
table_diameter = 3.0 * pot_diameter

##############################
# 4. Synthetic Shape Gens    #
##############################

def generate_box(num_points, x_min, x_max, y_min, y_max, z_min, z_max):
    x = np.random.uniform(x_min, x_max, num_points)
    y = np.random.uniform(y_min, y_max, num_points)
    z = np.random.uniform(z_min, z_max, num_points)
    return np.column_stack((x, y, z))

def generate_table(num_points, center, radius, z_value=0.0):
    angles = np.random.uniform(0, 2*np.pi, num_points)
    radii  = np.sqrt(np.random.uniform(0, 1, num_points)) * radius
    x = center[0] + radii * np.cos(angles)
    y = center[1] + radii * np.sin(angles)
    z = np.full(num_points, z_value)
    return np.column_stack((x, y, z))

def generate_random_plane(num_points, x_min, x_max, y_min, y_max, z_value, z_jitter=0.0):
    x = np.random.uniform(x_min, x_max, num_points)
    y = np.random.uniform(y_min, y_max, num_points)
    z = np.random.normal(loc=z_value, scale=z_jitter, size=num_points)
    return np.column_stack((x, y, z))

def generate_random_noise(num_points, region_box_min, region_box_max):
    x_min, y_min, z_min = region_box_min
    x_max, y_max, z_max = region_box_max
    x = np.random.uniform(x_min, x_max, num_points)
    y = np.random.uniform(y_min, y_max, num_points)
    z = np.random.uniform(z_min, z_max, num_points)
    return np.column_stack((x, y, z))

def add_gaussian_noise_to_points(points, sigma=0.002):
    """Add mild jitter to plant points (label=1)."""
    noise = np.random.normal(loc=0.0, scale=sigma, size=points.shape)
    return points + noise

##############################
# 5. Create Pot & Soil       #
##############################
pot_radius = pot_diameter / 2.0
pot_x_min  = -pot_radius
pot_x_max  = +pot_radius
pot_y_min  = -pot_radius
pot_y_max  = +pot_radius
pot_z_min  = 0.0
pot_z_max  = pot_height

num_pot_points = 3000
pot_points = generate_box(num_pot_points,
                          pot_x_min, pot_x_max,
                          pot_y_min, pot_y_max,
                          pot_z_min, pot_z_max)
pot_labels = np.zeros((pot_points.shape[0], 1))

soil_z_min = pot_z_max
soil_z_max = pot_z_max + soil_thickness
num_soil_points = 2000
soil_points = generate_random_plane(num_soil_points,
                                    x_min=pot_x_min, x_max=pot_x_max,
                                    y_min=pot_y_min, y_max=pot_y_max,
                                    z_value=pot_z_max,
                                    z_jitter=0.01 * pot_height)
soil_labels = np.zeros((soil_points.shape[0], 1))

##############################
# 6. Table Under the Pot     #
##############################
table_radius = table_diameter / 2.0
table_z = -0.05 * pot_height
num_table_points = 3000
table_points = generate_table(num_table_points,
                              center=(0.0, 0.0),
                              radius=table_radius,
                              z_value=table_z)
table_labels = np.zeros((table_points.shape[0], 1))

##############################
# 7. Position the Plant      #
##############################
def position_plant(pts, pot_center=(0,0), plant_base_z=pot_z_max):
    min_pt = np.min(pts, axis=0)
    max_pt = np.max(pts, axis=0)
    center_xy = (min_pt[:2] + max_pt[:2]) / 2.0
    min_z = min_pt[2]
    translate_xy = np.array(pot_center) - center_xy
    translate_z  = plant_base_z - min_z
    translation = np.array([translate_xy[0], translate_xy[1], translate_z])
    return pts + translation

##############################
# 8. Create Plant Noise      #
##############################
# (A) Mild sensor jitter on real plant
sigma = 0.002
plant_points_noisy = add_gaussian_noise_to_points(plant_points, sigma)

# (B) Position the noisy plant
plant_points_noisy = position_plant(plant_points_noisy,
                                    pot_center=(0, 0),
                                    plant_base_z=soil_z_max)

##############################
# 9. Generate "Scan Artifact"
##############################
def generate_scanning_artifacts(plant_pts, artifact_ratio=0.1, artifact_sigma=0.005):
    """
    Create scanning artifact points near the plant surface, labeled = 0.
    """
    num_plant = plant_pts.shape[0]
    num_artifacts = int(artifact_ratio * num_plant)
    
    seed_indices = np.random.choice(num_plant, size=num_artifacts, replace=False)
    seed_points = plant_pts[seed_indices, :]  # (num_artifacts, 3)
    
    offsets = np.random.normal(loc=0.0, scale=artifact_sigma, size=seed_points.shape)
    artifact_points = seed_points + offsets
    return artifact_points

artifact_ratio = 0.1   # 10% of plant points
artifact_sigma = 0.005 # e.g., 5 mm offsets
artifact_points = generate_scanning_artifacts(plant_points_noisy,
                                              artifact_ratio,
                                              artifact_sigma)
artifact_labels = np.zeros((artifact_points.shape[0], 1))

##############################
# 10. Random Noise in Scene  #
##############################
noise_min = np.array([
    pot_x_min - pot_radius,
    pot_y_min - pot_radius,
    table_z - 0.5 * pot_height
])
noise_max = np.array([
    pot_x_max + pot_radius,
    pot_y_max + pot_radius,
    soil_z_max + 0.5 * pot_height
])

num_scene_noise = 2000
scene_noise_points = generate_random_noise(num_scene_noise,
                                           region_box_min=noise_min,
                                           region_box_max=noise_max)
scene_noise_labels = np.zeros((scene_noise_points.shape[0], 1))

##############################
# 11. Combine All Points     #
##############################
non_plant_points = np.vstack([
    pot_points,
    soil_points,
    table_points,
    scene_noise_points,
    artifact_points
])
non_plant_labels = np.zeros((non_plant_points.shape[0], 1))

# Final dataset includes:
# - Real (noisy) plant points (label=1)
# - Synthetic pot/soil/table/scene noise + scanning artifacts (label=0)
all_points = np.vstack([plant_points_noisy, non_plant_points])
all_labels = np.vstack([plant_labels, non_plant_labels])

print("All points shape:", all_points.shape)
print("All labels shape:", all_labels.shape)


In [ ]:
##############################
# 12. Visualization          #
##############################
def visualize_points(points, labels, title="Plant/Non-Plant"):
    from mpl_toolkits.mplot3d import Axes3D
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    plant_mask = (labels.flatten() == 1)
    non_plant_mask = (labels.flatten() == 0)
    
    ax.scatter(points[non_plant_mask, 0],
               points[non_plant_mask, 1],
               points[non_plant_mask, 2],
               c='red', s=1, alpha=0.5, label='Non-Plant')
    ax.scatter(points[plant_mask, 0],
               points[plant_mask, 1],
               points[plant_mask, 2],
               c='green', s=5, alpha=0.8, label='Plant')
    
    ax.set_title(title)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    ax.legend()
    plt.show()

def visualize_open3d(points, labels, title="Plant/Non-Plant"):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    
    colors = np.zeros_like(points)
    plant_mask = (labels.flatten() == 1)
    non_plant_mask = (labels.flatten() == 0)
    colors[plant_mask] = [0, 1, 0]
    colors[non_plant_mask] = [1, 0, 0]
    
    pcd.colors = o3d.utility.Vector3dVector(colors)
    o3d.visualization.draw_geometries([pcd], window_name=title)

# Visualize
visualize_points(all_points, all_labels, "Plant with Synthetic Artifacts")
visualize_open3d(all_points, all_labels, "Plant/Non-Plant 3D View")

In [ ]:
##############################
# 13. Save Data (Optional)   #
##############################

def save_toy_dataset(points, labels, filename_points="toy_points.npy", filename_labels="toy_labels.npy"):
    np.save(filename_points, points)
    np.save(filename_labels, labels)
    print(f"Saved points to {filename_points}, labels to {filename_labels}")

save_toy_dataset(all_points, all_labels)